<a href="https://colab.research.google.com/github/sakshiyelmar/Deep-Learning-Assignment/blob/main/MNIST(CNN)_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [16]:
train=pd.read_csv('/content/drive/MyDrive/MNIST/train.csv')
test=pd.read_csv('/content/drive/MyDrive/MNIST/test.csv')

In [17]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [20]:
x_train = train.drop(columns=['label'], axis=1).astype('float32')
y_train = train['label'].astype('int32')
x_test = test.astype('float32')

x_train.shape, y_train.shape, x_test.shape

((42000, 784), (42000,), (28000, 784))

In [21]:
x_train = x_train.values.reshape(-1,28,28,1) / 255.0
x_test = x_test.values.reshape(-1,28,28,1) / 255.0

In [22]:
x_train.shape,y_train.shape,x_test.shape

((42000, 28, 28, 1), (42000,), (28000, 28, 28, 1))

In [23]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 10)
y_train.shape

(42000, 10)

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

((33600, 28, 28, 1), (8400, 28, 28, 1), (33600, 10), (8400, 10))

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense,Dropout, BatchNormalization)
from tensorflow.keras.callbacks import EarlyStopping

In [26]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
    BatchNormalization(),
    Conv2D(32, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),

    Conv2D(64, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 26, 26, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 24, 24, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 12, 12, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 12, 12, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 10, 10, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 10, 10, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 8, 8, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 198,378 (774.91 KB)

 Trainable params: 197,930 (773.16 KB)

 Non-trainable params: 448 (1.75 KB)

In [28]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=15,
                    batch_size=128,
                    verbose=1,callbacks=early_stopping)

Epoch 1/15
263/263 ━━━━━━━━━━━━━━━━━━━━ 99s 359ms/step - accuracy: 0.7104 - loss: 0.9685 - val_accuracy: 0.1119 - val_loss: 6.7402
Epoch 2/15
263/263 ━━━━━━━━━━━━━━━━━━━━ 93s 355ms/step - accuracy: 0.9548 - loss: 0.1459 - val_accuracy: 0.8514 - val_loss: 0.4953
Epoch 3/15
263/263 ━━━━━━━━━━━━━━━━━━━━ 140s 349ms/step - accuracy: 0.9681 - loss: 0.1059 - val_accuracy: 0.9846 - val_loss: 0.0541
Epoch 4/15
263/263 ━━━━━━━━━━━━━━━━━━━━ 142s 350ms/step - accuracy: 0.9762 - loss: 0.0754 - val_accuracy: 0.9882 - val_loss: 0.0402
Epoch 5/15
263/263 ━━━━━━━━━━━━━━━━━━━━ 142s 350ms/step - accuracy: 0.9803 - loss: 0.0630 - val_accuracy: 0.9877 - val_loss: 0.0441
Epoch 6/15
263/263 ━━━━━━━━━━━━━━━━━━━━ 143s 354ms/step - accuracy: 0.9827 - loss: 0.0558 - val_accuracy: 0.9886 - val_loss: 0.0374
Epoch 7/15
263/263 ━━━━━━━━━━━━━━━━━━━━ 141s 351ms/step - accuracy: 0.9855 - loss: 0.0471 - val_accuracy: 0.9905 - val_loss: 0.0356
Epoch 8/15
263/263 ━━━━━━━━━━━━━━━━━━━━ 141s 347ms/step - accuracy: 0.9865 - l

In [32]:
test_loss, test_accuracy = model.evaluate(x_val, y_val, verbose=1)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

263/263 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9910 - loss: 0.0270
Test Loss: 0.02778918854892254
Test Accuracy: 0.9920238256454468


In [33]:
predictions = model.predict(x_test)
test_labels = np.argmax(predictions, axis=1)

875/875 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step


In [34]:
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(test_labels) + 1),
    "Label": test_labels
})

In [36]:
submission.to_csv('submission.csv', index=False)

In [37]:
print(submission)

       ImageId  Label
0            1      2
1            2      0
2            3      9
3            4      9
4            5      3
...        ...    ...
27995    27996      9
27996    27997      7
27997    27998      3
27998    27999      9
27999    28000      2

[28000 rows x 2 columns]
